## FCLA/FNLA Fast.ai Numerical/Computational Linear Algebra

### Lecture 3: New Perspectives on NMF, Randomized SVD
Notes / In-Class Questions

WNixalo - 2018/2/8

Question on section: [Truncated SVD](http://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/2.%20Topic%20Modeling%20with%20NMF%20and%20SVD.ipynb#More-Details)

Given A: `m` x `n` and Q: `m` x `r`; is Q the identity matrix?

A≈QQTA

In [1]:
import torch
import numpy as np

In [27]:
Q = np.eye(3)
print(Q)
print(Q.T)
print(Q @ Q.T)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [26]:
# construct I matrix
Q = torch.eye(3)

# torch matrix multip
# torch.mm(Q, Q.transpose)
Q @ torch.t(Q)


 1  0  0
 0  1  0
 0  0  1
[torch.FloatTensor of size 3x3]

So if A is *approx equal* to Q•Q.T•A .. but *not* equal.. then Q is **not** the identity, but is very close to it.

Oh, right. Q: m x r, **not** m x m... 

If both the columns and rows of Q had been orthonormal, then it would have been the Identity, but only the columns (r) are orthonormal.

Q is a tall, skinny matrix.


---

AW gives range(A). AW has far more rows than columns ==> in practice these columns are approximately orthonormal (v.unlikely to get lin-dep cols when choosing random values).

QR decomposition is foundational to Numerical Linear Algebra.

Q consists of orthonormal columns, R is upper-triangular.

**Calculating Truncated-SVD:**

1\. Compute approximation to range(A). We want Q with r orthonormal columns such that $$A\approx QQ^TA$$

2\. Construct $B = Q^T A$, which is small ($r\times n$)

3\. Compute the SVD of $B$ by standard methods (fast since $B$ is smaller than $A$): $B = S\, Σ V^T$

4\. Since: $$A \approx QQ^TA = Q(S \, ΣV^T)$$ if we set $U = QS$, then we have a low rank approximation $A \approx UΣV^T$.

**How to choose $r$?**

If we wanted to get 5 cols from a matrix of 100 cols, (5 topics). As a rule of thumb, let's go for 15 instead. You don't want to explicitly pull exactly the amount you want due to the randomized component being present, so you add some buffer.

Since our projection is approximate, we make it a little bigger than we need.

**Implementing Randomized SVD:**

First we want a randomized range finder.

In [40]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt

%matplotlib inline
np.set_printoptions(suppress=True)

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
# newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_train.data).todense() # (documents, vocab)

vocab = np.array(vectorizer.get_feature_names())

In [41]:
num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [35]:
# computes an orthonormal matrix whose range approximates the range of A
# power_iteration_normalizer can be safe_sparse_dot (fast but unstable), LU (imbetween), or QR (slow but most accurate)
def randomized_range_finder(A, size, n_iter=5):
    # randomly init our Mat to our size; size: num_cols
    Q = np.random.normal(size=(A.shape[1], size))
    
    # LU decomp (lower triang * upper triang mat)
    # improves accuracy & normalizes
    for i in range(n_iter):
        Q, _ = linalg.lu(A @ Q, permute_l=True)
        Q, _ = linalg.lu(A.T @ Q, permute_l=True)
    
    # QR decomp on A & Q
    Q, _ = linalg.qr(A @ Q, mode='economic')
    return Q

Randomized SVD method:

In [36]:
def randomized_svd(M, n_components, n_oversamples=10, n_iter=4):
    
    # number of random columns we're going to create is the number of 
    # columns we want + number of oversamples (extra buffer)
    n_random = n_components + n_oversamples
    
    Q = randomized_range_finder(M, n_random, n_iter)
    
    # project M to the (k + p) dimensional space using basis vectors
    B = Q.T @ M
    
    # compute SVD on the thin matrix: (k + p) wide
    Uhat, s, V = linalg.svd(B, full_matrices=False)
    del B
    U = Q @ Uhat
    
    # return the number of components we want from U, s, V
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [42]:
%time u, s, v = randomized_svd(vectors, 5)

CPU times: user 4.63 s, sys: 1.89 s, total: 6.52 s
Wall time: 4.12 s


In [43]:
u.shape, s.shape, v.shape

((2034, 5), (5,), (5, 26576))

In [44]:
show_topics(v)

['jpeg image edu file graphics images gif data',
 'jpeg gif file color quality image jfif format',
 'space jesus launch god people satellite matthew atheists',
 'jesus god matthew people atheists atheism does graphics',
 'jpeg graphics space pub edu ray mail send']

Computational Complexity for a M`x`N matrix in SVD is $M^2N+N^3$, so Randomized (Truncated?) SVD is a *massive* improvement.